In [1]:
import cv2
import numpy as np

In [2]:
win_name = 'scan'
src = cv2.imread('./sample1.jpg') # 이미지 파일 읽기
img = src.copy()


In [3]:
resize = cv2.resize(img, dsize=(0, 0), fx=0.2, fy=0.2, interpolation=cv2.INTER_AREA)
cv2.imshow("조절", resize)
cv2.waitKey(0) # param 시간동안 키보드 입력 대기 - 0 인경우 무한대기
cv2.destroyAllWindows()

In [4]:
gray = cv2.cvtColor(resize, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3, 3), 0) # 이미지를 흐리게 처리함 (noise 제거를 위해 사용)
edged = cv2.Canny(gray, 75, 250) # edged를 검출하는 함수 (img, minVal, maxVal)
cv2.imshow('gray', edged)  # 윈도우 창에 보여주기 (윈도우창 title, 이미지)
cv2.waitKey(0) # param 시간동안 키보드 입력 대기 - 0 인경우 무한대기
cv2.destroyAllWindows()

In [6]:
# 컨투어 찾기
contours, hierarchy = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
len(contours)

3

In [8]:
contours[0]

array([[[603, 778]],

       [[602, 779]],

       [[590, 779]],

       [[589, 780]],

       [[566, 780]],

       [[565, 781]],

       [[546, 781]],

       [[545, 782]],

       [[536, 782]],

       [[535, 783]],

       [[527, 783]],

       [[526, 782]],

       [[525, 783]],

       [[514, 783]],

       [[513, 784]],

       [[483, 784]],

       [[482, 785]],

       [[465, 785]],

       [[464, 786]],

       [[459, 786]],

       [[458, 785]],

       [[457, 785]],

       [[456, 786]],

       [[445, 786]],

       [[444, 787]],

       [[440, 787]],

       [[439, 786]],

       [[436, 786]],

       [[435, 787]],

       [[433, 787]],

       [[432, 786]],

       [[430, 786]],

       [[429, 787]],

       [[400, 787]],

       [[399, 788]],

       [[394, 788]],

       [[393, 787]],

       [[391, 787]],

       [[390, 788]],

       [[352, 788]],

       [[351, 789]],

       [[350, 788]],

       [[322, 788]],

       [[321, 789]],

       [[293, 789]],

       [[2

In [6]:
draw = resize.copy()
cv2.drawContours(draw,  contours, -1, (0, 255, 0))
cv2.imshow("contours", draw)
cv2.waitKey(0)
cv2.destroyAllWindows()

외곽선 관련 함수
- cv2.arcLength() : 외곽선의 길이를 반환
- cv2.contourArea() : 외곽선이 감싸는 영역의 면적을 반환
- cv2.boundingRect() : 주어진 점을 감싸는 최소 크기의 사각형(바운딩 박스)를 반환
- cv2.minEnclosingCircle() : 주어진 점을 감싸는 최소 크기의 원을 반환
- cv2.minAreaRect() : 주어진 점을 감싸는 최소 크기의 회전된 사각형을 반환
- cv2.approxPolyDP() : 외곽선을 근사화(단순화)
- cv2.fitEllipse() : 주어진 점에 적합한 타원을 반환
- cv2.fitLine() : 주어진 점에 적합한 직선을 반환
- cv2.isContourConvex() : 컨벡스인지를 검사
- cv2.convexHull() : 주어진 점으로부터 컨벤스 헐을 반환
- cv2.convexityDefects() : 주어진 점과 컨벡스 헐로부터 컨벡스 디펙트를 반환

참조 사이트
 - https://deep-learning-study.tistory.com/232
 

In [7]:
# 컨투어들 중에 영역 크기 순으로 정렬
contours = sorted(contours, key=cv2.contourArea, reverse=True)[:5]
for c in contours:
    peri = cv2.arcLength(c, True) # 외곽선 길이
    # print(peri)
    verticles = cv2.approxPolyDP(c, 0.02 * peri, True) # 외곽선 근사화
    if len(verticles) == 4 : 
        break
pts = verticles.reshape(4, 2) # 배열을 4 * 2 크기로 조정

In [8]:
print(pts)
print(pts.sum(axis=1))
print(np.diff(pts, axis=1))
print("좌상단", pts[np.argmin(pts.sum(axis=1))])

[[512  68]
 [ 50  73]
 [ 38 748]
 [552 736]]
[ 580  123  786 1288]
[[-444]
 [  23]
 [ 710]
 [ 184]]
좌상단 [50 73]


In [9]:
# 각각의 좌표 찾기
sumXY = pts.sum(axis=1)
diff = np.diff(pts, axis=1)

topLeft = pts[np.argmin(sumXY)]
bottomRight = pts[np.argmax(sumXY)]
topRight = pts[np.argmin(diff)]
bottomLeft = pts[np.argmax(diff)]

np.float32([topLeft, topRight, bottomRight, bottomLeft])

array([[ 50.,  73.],
       [512.,  68.],
       [552., 736.],
       [ 38., 748.]], dtype=float32)

In [10]:
# 사진을 변환할 때 사용할 서류의 높이
widthTop = abs(topRight[0] - topLeft[0])
widthBottom = abs(bottomRight[0] - bottomLeft[0])
heightRight = abs(topRight[1] - bottomRight[1])
heightLeft = abs(topLeft[1] - bottomLeft[1])
print(widthBottom, widthTop, heightLeft, heightRight)

width = max([widthTop, widthBottom])
height = max([heightRight, heightLeft])

514 462 675 668


In [11]:
pts1 = np.float32([topLeft, topRight, bottomRight, bottomLeft])
pts2 = np.float32([[0,0], [width, 0], [width, height], [0, height]])

In [23]:
matrix = cv2.getPerspectiveTransform(pts1, pts2) # 좌표를 변환하기 위해 사용할 변환행렬
result = cv2.warpPerspective(resize, matrix, (width, height)) # 이미지 변환(변환행렬 적용)
cv2.imshow("result", result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [13]:
import easyocr

In [24]:
reader = easyocr.Reader(['ko', 'en'], gpu=True)
txt = reader.readtext(result)
print(txt)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


[([[195, 89], [335, 89], [335, 133], [195, 133]], '정책실천', 0.9139558672904968), ([[122, 128], [406, 128], [406, 160], [122, 160]], '반시 국민의미래가합니다', 0.6685272033810605), ([[197, 179], [327, 179], [327, 199], [197, 199]], '국민의미래 10대 공약', 0.7629466924434477), ([[149, 207], [307, 207], [307, 223], [149, 223]], '피망치고 안전한 교육 물봄 환경구;', 0.16873579856446477), ([[89, 215], [119, 215], [119, 231], [89, 231]], '정촉1', 0.28441463239667303), ([[216, 222], [262, 222], [262, 230], [216, 230]], '프고표 극: J', 0.06103644963269899), ([[149, 221], [239, 221], [239, 241], [149, 241]], '쓸t하다 서한 집변청골고교', 0.0002787984775832519), ([[250, 230], [278, 230], [278, 238], [250, 238]], '요속라오', 0.0017270867247134447), ([[149, 247], [295, 247], [295, 261], [149, 261]], '자입준비 정년의사회첫?음 지원', 0.13645117948199886), ([[89, 255], [121, 255], [121, 271], [89, 271]], '정2', 0.68356559760284), ([[151, 261], [193, 261], [193, 281], [151, 281]], '물취시보리다', 0.0021188467890717808), ([[192, 270], [224, 270], [224, 278], [192, 278]], '우무 기',

In [22]:
for x in txt :
    print(x[1])

정책실천
반시 국민의미래가합니다
국민의미래 10대 공약
피망치고 안전한 교육 물봄 환경구;
정촉1
프고표 극: J
쓸t하다 서한 집변청골고교
요속라오
자입준비 정년의사회첫?음 지원
정2
물취시보리다
우무 기
@금로번 "실_바물구r
한부모가정 위기임산부 아이기우기 행복한 대한민국만큼기
정책 3
:털다 디
한주퇴자락의a 적발핸기앉
오Vouup - 나가 그녀틱
장대인의 사회적 걱차해소
정촉94
0: 기긴 주7;
뵙다말
'n신__관
따뜻한 동행 국한이튿주민 두작기 지원
정석5
벌손다#[필~
산미준하
UN 저살 굽사 스신
소외되논 이웃이 엎도록 모두가 잡사논 동행사회 만줄기
정책 6
달갈=
라어기비
'고소자은없다 버리리회
군장빛둥올 위관 안전한 환경조성 및 근무관경 개선
정척7
조플증리터사카되터
특수탕 지: 인계
제복공무원 처우 및 근무1경 개선
정책 8
수하 
다u 7
'트롭선다삼 시길
중소기업 근로환경 개선
정책 9
어너이가 '
욱절서버터쪽면
J리H5 대'간
[간현심 빗 그전 린하 듭 근고신리 더자서 스
지속기농한 미래사회름 위한 기후 위기대용
정책 10
버터 과어
도가뻐
'효@국 듭보스
